In [5]:
%load_ext rqalpha

The rqalpha extension is already loaded. To reload it, use:
  %reload_ext rqalpha


In [6]:
%%rqalpha -h
""

Usage: ipykernel_launcher.py [OPTIONS]

  Start to run a strategy

Options:
  -h, --help                      Show this message and exit.
  -d, --data-bundle-path PATH
  -f, --strategy-file PATH
  -s, --start-date DATE
  -e, --end-date DATE
  -bm, --benchmark TEXT
  -mm, --margin-multiplier FLOAT
  -a, --account TEXT...           set account type with starting cash
  --position TEXT                 set init position
  -fq, --frequency [1d|1m|tick]
  -rt, --run-type [b|p|r]
  --resume
  --source-code TEXT
  -l, --log-level [verbose|debug|info|error|none]
  --disable-user-system-log       disable user system log stdout
  --disable-user-log              disable user log stdout
  --logger TEXT...                config logger, e.g. --logger system_log
                                  debug
  --locale [cn|en]
  --extra-vars TEXT               override context vars
  --enable-profiler               add line profiler to profile your strategy
  --dividend-reinvestment         enable dividend r

In [3]:
%%rqalpha -s 20100101 -e 20170505 -p -bm 000001.XSHG --account stock 100000


# 在这个方法中编写任何的初始化逻辑。context对象将会在你的算法策略的任何方法之间做传递。
import numpy as np
import pandas as pd


def init(context):
    # 在context中保存全局变量
    #移动平均价格周期m1,m2,m3,m4
    context.m1=3
    context.m2=6
    context.m3=12
    context.m4=24
    context.number=10 #取前10只股票进行交易
    #正成交量指标
    
# before_trading此函数会在每天策略交易开始前被调用，当天只会被调用一次
def before_trading(context):
    pass

def get_stock(context):
    
    pass
    
def BBI(context):
    #建立BBI指标
    pass

def PVI(context,bar_dict):
    
    pass
    
        
    
    
    

# 你选择的证券的数据更新将会触发此段逻辑，例如日或分钟历史数据切片或者是实时数据切片更新
def handle_bar(context, bar_dict):
    # 开始编写你的主要的算法逻辑

    # bar_dict[order_book_id] 可以拿到某个证券的bar信息
    # context.portfolio 可以拿到现在的投资组合信息

    # 使用order_shares(id_or_ins, amount)方法进行落单

    # TODO: 开始编写你的算法吧！
    
    #开始进行交易
    #建立BBI多空指数
    data=get_fundamentals(query(fundamentals.income_statement.revenue,fundamentals.financial_indicator.earnings_per_share,fundamentals.eod_derivative_indicator.pe_ratio).filter(fundamentals.income_statement.revenue>0).filter(fundamentals.financial_indicator.earnings_per_share>0).filter(fundamentals.eod_derivative_indicator.pe_ratio>0))
    data=data.T
    #按照营业收入排序，打分
    data=data.sort_values(['revenue'],ascending=False)
    data['revenue_rank']=np.arange(len(data),0,-1)
    #按照市盈率排序，打分
    data=data.sort_values(['pe_ratio'],ascending=True)
    data['pe_rank']=np.arange(len(data),0,-1)
    #按照每股收益排序，打分
    data=data.sort_values(['earnings_per_share'],ascending=False)
    data['eps_rank']=np.arange(len(data),0,-1)
    
    #计算总分，进行排序，
    data['total_number']=data['revenue_rank']+data['pe_rank']+data['eps_rank']
    data=data.sort_values(['total_number'],ascending=False)
    stock=data.head(context.number).index #筛选出需要购买的10只股票
    stock=set(stock)
    
    #BBI
    BBI={}
    for i in stock:
        price_m1=sum(history_bars(i,context.m1,'1d','close'))/context.m1
        price_m2=sum(history_bars(i,context.m2,'1d','close'))/context.m2
        price_m3=sum(history_bars(i,context.m3,'1d','close'))/context.m3
        price_m4=sum(history_bars(i,context.m4,'1d','close'))/context.m4
        
        value=(price_m1+price_m2+price_m3+price_m4)/4
        BBI[i]=value
    
    #PVI
    PVI={}
    for i in stock:
        price_today=bar_dict[i]
        price=history_bars(i,10,'1d','close')
        volume=history_bars(i,10,'1d','volume')
        if bar_dict[i].volume>volume[0]:
            PVI[i]=100*bar_dict[i].volume/volume[0]
        else:
            PVI[i]=100
    
        
    
    for i in stock:
        avg=sum(history_bars(i,5,'1d','close'))/5
        if bar_dict[i].open>BBI[i] and PVI[i]>avg:
            order_target_percent(i,0.1)
        
        if bar_dict[i].open<BBI[i] and PVI[i]<avg:
            order_target_percent(i,0)
            
        
    
    
    

# after_trading函数会在每天交易结束后被调用，当天只会被调用一次
def after_trading(context):
    pass

AttributeError: module 'better_exceptions' has no attribute 'format_traceback'

In [7]:
%%rqalpha -s 20100101 -e 20170505 -p -bm 000001.XSHG --account stock 100000
# 可以自己import我们平台支持的第三方python模块，比如pandas、numpy等。

import talib
import numpy as np
import pandas as pd

# 在这个方法中编写任何的初始化逻辑。context对象将会在你的算法策略的任何方法之间做传递。
def init(context):
    
    #从金融分类中选取部分
    context.stocks =     sector("Financials")[1:20]

    #设定每只股票投资占总资金的投资份额
    context.avg_pct = 1.0 / len(context.stocks) - 0.01


# 你选择的证券的数据更新将会触发此段逻辑，例如日或分钟历史数据切片或者是实时数据切片更新
    update_universe(context.stocks)
    
def handle_bar(context, bar_dict):

    
    # 历史数据 - 为了之后运算使用
    high = history(30, '1d', 'high')
    low = history(30, '1d', 'low')
    close = history(30, '1d', 'close')
    

    for stock in context.stocks:
        curPosition = context.portfolio.positions[stock].quantity
        
        slowk, slowd = talib.STOCH(high[stock].values,
                                   low[stock].values,
                                   close[stock].values,
                                   fastk_period=5,
                                   slowk_period=3,
                                   slowk_matype=0,
                                   slowd_period=3,
                                   slowd_matype=0)

        # 拿到slowk 和slowd最近的一个值
        slowk = slowk[-1]
        slowd = slowd[-1]
        
        # 如果 slowk <10 或者 slowd 小于 20, 那么股票处于超卖阶段，我们选择买入
        # in the portfolio.
        if (slowk < 10 or slowd < 20) and curPosition <= 0:
            order_target_percent(stock, context.avg_pct)
        
        # 如果 slowk 或者 slowd 小于 10, 那么股票处于超买阶段，我们选择卖出
 
        elif (slowk > 90 or slowd > 80) and curPosition > 0:
            order_target_percent(stock, 0)

#该策略使用TALIB里面的STOCH 技术指标函数去决定买入和卖出点

#这里面slowk,slowd的相当于kdj里面的KD线

AttributeError: module 'better_exceptions' has no attribute 'format_traceback'

In [8]:
%%rqalpha -s 20100101 -e 20170505 -p -bm 000001.XSHG --account stock 100000
# 可以自己import我们平台支持的第三方python模块，比如pandas、numpy等。
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

# 在这部分编写技术分析模块
def RSI(N1=6, N2=12, N3=24):
    """
    RSI 相对强弱指标
    """
    LC = REF(CLOSE, 1)
    RSI1 = SMA(MAX(CLOSE - LC, 0), N1, 1) / SMA(ABS(CLOSE - LC), N1, 1) * 100
    RSI2 = SMA(MAX(CLOSE - LC, 0), N2, 1) / SMA(ABS(CLOSE - LC), N2, 1) * 100
    RSI3 = SMA(MAX(CLOSE - LC, 0), N3, 1) / SMA(ABS(CLOSE - LC), N3, 1) * 100

    return RSI1, RSI2, RSI3

# 在这个方法中编写任何的初始化逻辑。context对象将会在你的算法策略的任何方法之间做传递。
def init(context):
    reg_indicator('RSI', RSI, '1d', win_size=40)
    context.buy = []
    context.sell = []
    context.hold = []
    context.s_sell = []

# before_trading此函数会在每天策略交易开始前被调用，当天只会被调用一次
def before_trading(context):
    context.buy = []
    context.sell = []
    context.hold = []
    stocks,_ = get_all_stocks(context)
    
    for stock in context.portfolio.positions.keys():
        RSI1,RSI2,RSI3 = get_indicator(stock, 'RSI')
        if RSI1>80 and REF(RSI1,1) > REF(RSI2,1) and RSI1 < RSI2:
            context.sell.append(stock)
        else:
            context.hold.append(stock)
    
    for stock in context.s_sell:
        if stock not in context.portfolio.positions.keys():
            context.s_sell.remove(stock)
            
    if len(context.hold) >= 10:
        return None
        
    for stock in stocks:
        RSI1,RSI2,RSI3 = get_indicator(stock, 'RSI')
        if RSI1<20 and REF(RSI1,1) < REF(RSI2,1) and RSI1 > RSI2:
            context.buy.append(stock)
            if stock in context.sell:
                context.sell.remove(stock)


# 你选择的证券的数据更新将会触发此段逻辑，例如日或分钟历史数据切片或者是实时数据切片更新
def handle_bar(context, bar_dict):
    for stock in context.sell:
        order = order_target_percent(stock,0)
        if order.unfilled_quantity != 0:
            context.s_sell.append(stock)
    for stock in context.s_sell:
        order_target_value(stock,0)
    
    if len(context.hold)+len(context.buy) == 0:
        return None
    weight = 1/(len(context.hold)+len(context.buy))
    
    for stock in context.hold:
        order_target_percent(stock,weight)
    for stock in context.buy:
        order_target_percent(stock,weight)

# after_trading函数会在每天交易结束后被调用，当天只会被调用一次
def after_trading(context):
    pass

def get_all_stocks(context):
    all_stocks = all_instruments("CS").order_book_id
    will_end = []
    trade = []
    for stock in all_stocks:
        ins = instruments(stock)
        if ins is None:
            pass
        else:
            start = ins.listed_date
            end = ins.de_listed_date
            if (start-context.now).days < 0:
                if 0< (end - context.now).days <30:
                    will_end.append(stock)
                elif 30 < (end - context.now).days and is_suspended(stock) == False:
                    trade.append(stock)
    #print(len(trade),len(will_end))
    return trade,will_end

AttributeError: module 'better_exceptions' has no attribute 'format_traceback'

In [9]:
%%rqalpha -s 20100101 -e 20170505 -p -bm 000001.XSHG --account stock 100000
# 可以自己import我们平台支持的第三方python模块，比如pandas、numpy等。
import pandas
import numpy
import talib

# 在这个方法中编写任何的初始化逻辑。context对象将会在你的算法策略的任何方法之间做传递。
def init(context):
    # 在context中保存全局变量
    scheduler.run_weekly(operate, tradingday=2)

# before_trading此函数会在每天策略交易开始前被调用，当天只会被调用一次
def before_trading(context):
    pass


# 你选择的证券的数据更新将会触发此段逻辑，例如日或分钟历史数据切片或者是实时数据切片更新
def handle_bar(context, bar_dict):
    # 开始编写你的主要的算法逻辑

    # bar_dict[order_book_id] 可以拿到某个证券的bar信息
    # context.portfolio 可以拿到现在的投资组合信息

    # 使用order_shares(id_or_ins, amount)方法进行落单

    # TODO: 开始编写你的算法吧！
    pass


# after_trading函数会在每天交易结束后被调用，当天只会被调用一次
def after_trading(context):
    pass

def operate(context,stock):
    #初始化信息
    context.to_buy = []
    context.to_sell = []
    #提取市场n个股票
    context.num = 1000
    context.fundamental_df = get_fundamentals(
        query(
        ).limit(
            context.num
        )    
    )
    context.stocks = context.fundamental_df.columns.values
    print("提取市场股票数量为：")
    print(len(context.stocks))
    print('\n')
    
    
    #计算MACD指标,并构建股票池
    for stock in context.stocks:
        high = history_bars(stock, 20, '1d', 'high')
        low = history_bars(stock, 20, '1d', 'low')
        close = history_bars(stock, 20, '1d', 'close')
        K,D = talib.STOCH(high, low, close, fastk_period=9, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    
        #判断是否符合MACD买入指标，并添加入股票池
        if K[-1] < 10 or K[-1] < 10:
            context.to_buy.append(stock)
        if K[-1] > 89 or K[-1] > 90:
            context.to_sell.append(stock)

        
    #清仓不在股票池的股票
    for stock in context.portfolio.positions:
            if stock in context.to_sell:
                order_target_percent(stock,0)
        
    #计算股票池买入权重，并买入。
    if len(context.to_buy) == 0:
            weight = 0
    else:
            weight = 0.99 / (len(context.to_buy) + len(context.portfolio.positions))
            
    for stock in context.to_buy:
            order_target_percent(stock, weight)
    
    print("股票池个数为：")
    print(len(context.to_buy))
    print('\n')
            

AttributeError: module 'better_exceptions' has no attribute 'format_traceback'

In [10]:
%%rqalpha -s 20100101 -e 20170505 -p -bm 000001.XSHG --account stock 100000

import talib as ta
import pandas as pd
import numpy as np
import time
import datetime

def convert_date(sDate,iDelta=0):#日期转换
    d=time.strptime(sDate,'%Y-%m-%d')
    dt=datetime.date(d.tm_year,d.tm_mon,d.tm_mday)+datetime.timedelta(iDelta)
    return dt
def SMA(values,N,M):#计算通达信的SMA，values为array
    #通达信中SMA计算公式：SMA(x(n),N,M)=(M/N)*∑(k=0,30)(((N-M)/N)^k)*x(k).其中N为平均化周期，M为权重，一般取1.k越打越精确，一般取到30足够
    if len(values)<30:return -1
    k=np.arange(29,-1,-1)
    x=values[len(values)-31:len(values)-1]
    df1=pd.DataFrame()
    df1['x']=x
    df1['k']=k
    df1['SMA']=df1['x']*pow(((N-M)/N),df1['k'])
    #logger.info(df1)
    SMA=(M/N)*df1['SMA'].sum()
    return SMA
def kd_index(close,high,low,kPeriod=9,SlowKPeriod=3,SlowDPeriod=3):
    #取参数，对应通达信中变量名
    N=kPeriod
    M1=SlowKPeriod
    M2=SlowDPeriod
    M=1#SMA权重M
    df=pd.DataFrame(data=close,columns=['close'])
    df['close']=close
    df['high']=high
    df['low']=low
    for i in range(N,len(df.index)):
        df.ix[i,'highest']=max(df.ix[i-N+1:i,'high'])
        df.ix[i,'lowest']=min(df.ix[i-N+1:i,'low'])
    df['fastk']=(df['close']-df['lowest'])/(df['highest']-df['lowest'])*100
    #通达信中SMA计算公式：SMA(x(n),N,M)=(M/N)*∑(k=0,30)(((N-M)/M)^k)*x(k).其中N为平均化周期，M为权重，一般取1.k越打越精确，一般取到30足够
    for i in range(len(df.index)-30,len(df.index)):
        df.ix[i,'slowk']=SMA(df.ix[i-30:i+1,'fastk'].values,3,1)
    df.ix[len(df.index)-1,'slowd']=SMA(df.ix[len(df.index)-32:len(df.index),'slowk'].values,3,1)
    return df.ix[len(df.index)-1,'slowk'],df.ix[len(df.index)-1,'slowd']
    #return 10,10
def get_kd_signal(stock,bar_dict,kPeriod=12,SlowKPeriod=3,SlowDPeriod=3):
    period=2*(kPeriod+60)#为保证取得足够数据，向前取两个(k+N)日期
    close=history(period,'1d','close')[stock].values
    high=history(period,'1d','high')[stock].values
    low=history(period,'1d','low')[stock].values
    slowk,slowd=kd_index(close,high,low,kPeriod,SlowKPeriod,SlowDPeriod)
    #print(slowk,slowd)
    if slowk>80 and slowd>80:return 'sell'
    if slowk<20 and slowd<20:return 'buy'
def init(context):
    context.s1 = "510050.XSHG"
    context.benchmark="510050.XSHG"
def before_trading(context, bar_dict):
    pass

def handle_bar(context, bar_dict):
    result=get_kd_signal(context.s1,bar_dict,31,5,2)
    if result=='buy' and context.portfolio.positions[context.s1].quantity==0:
        order_target_percent(context.s1,0.9)
    if result=='sell' and context.portfolio.positions[context.s1].quantity>0:
        order_target_percent(context.s1,0)
        

2010-01-04 WARN [Deprecated]在before_trading函数中，第二个参数bar_dict已经不再使用了。
2010-01-04 WARN [abandon] context.benchmark 已经不再使用


AttributeError: module 'better_exceptions' has no attribute 'format_traceback'

In [11]:
!pip list --format=columns | grep numpy

numpy                         1.14.0   


In [13]:
%%rqalpha -s 20100101 -e 20170505 -p -bm 000001.XSHG --account stock 100000

import funcat as fc
def init(context):
    # 实时打印日志
    logger.info("RunInfo: {}".format(context.run_info))
# before_trading此函数会在每天策略交易开始前被调用，当天只会被调用一次
def before_trading(context):
    fundamentals_df=get_fundamentals(
    query(
    fundamentals.eod_derivative_indicator.market_cap_2)) 
    context.stocks=list(fundamentals_df.columns.values)
    #logger.info("context.stocks"+str(context.stocks))
    print('股票总数：',len(context.stocks))
    print('------')
    for stock in context.stocks:
        print(stock,C)
    pass

# 你选择的证券的数据更新将会触发此段逻辑，例如日或分钟历史数据切片或者是实时数据切片更新
def handle_bar(context, bar_dict):
    pass

2010-01-04 INFO RunInfo: RunInfo({'matching_type': MATCHING_TYPE.CURRENT_BAR_CLOSE, 'stock_starting_cash': 100000.0, 'run_type': RUN_TYPE.BACKTEST, 'benchmark': '000001.XSHG', 'slippage': 0, 'future_starting_cash': 0, 'margin_multiplier': 1, 'end_date': datetime.date(2017, 5, 5), 'start_date': datetime.date(2010, 1, 4), 'commission_multiplier': 1, 'frequency': '1d'})


AttributeError: module 'better_exceptions' has no attribute 'format_traceback'